In [1]:
import qiskit
import sys
sys.path.append('../../../aqcel_dev/aqcel_dev')
import aqcel_dev as icepp
import qiskit.quantum_info as qi

# QASM形式のファイルからQuantumCircuitを読み込む
with open("data/circuit.qasm_NQPSB", "r") as f:
    qasm_str_NQPSB = f.read()
    # print(qasm_str_NQPSB)
with open("data/circuit.qasm_NQPSA", "r") as f:
    qasm_str_NQPSA = f.read()

Nstep_before_AQCEL = qiskit.QuantumCircuit.from_qasm_str(qasm_str_NQPSB)
Nstep_after_AQCEL  = qiskit.QuantumCircuit.from_qasm_str(qasm_str_NQPSA)

# QASM形式のファイルからQuantumCircuitを読み込む
with open("data/circuit.qasm_OQPSB", "r") as f:
    qasm_str_OQPSB = f.r
    ead()
with open("data/circuit.qasm_OQPSA", "r") as f:
    qasm_str_OQPSA = f.read()

Optimized_before_AQCEL = qiskit.QuantumCircuit.from_qasm_str(qasm_str_OQPSB)
Optimized_after_AQCEL  = qiskit.QuantumCircuit.from_qasm_str(qasm_str_OQPSA)

QasmError: "Duplicate declaration for qreg 'p' at line 4, file .\nPrevious occurrence at line 24, file /usr/local/lib/python3.8/dist-packages/qiskit/qasm/libs/qelib1.inc"

In [ ]:
# qc1←→qc1'

statevector1 = icepp.simulator(Nstep_before_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Nstep_after_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm2.probabilities_dict()

print(f"Fsim qc1←→qc1' = {qi.state_fidelity(dm1,dm2)}")

In [ ]:
# qc2←→qc2'

statevector1 = icepp.simulator(Optimized_before_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Optimized_after_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[6, 7, 8, 9, 10])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
dm2.probabilities_dict()

print(f"Fsim qc2←→qc2' = {qi.state_fidelity(dm1,dm2)}")

In [ ]:
# qc1←→qc2

statevector1 = icepp.simulator(Nstep_before_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Optimized_before_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
dm2.probabilities_dict()

print(f"Fsim qc1←→qc2 = {qi.state_fidelity(dm1,dm2)}")

In [ ]:
# qc1'←→qc2'

statevector1 = icepp.simulator(Nstep_after_AQCEL).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(Optimized_after_AQCEL).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[6, 7, 8, 9, 10])
dm2.probabilities_dict()

print(f"Fsim qc1'←→qc2' = {qi.state_fidelity(dm1,dm2)}")

In [1]:
from qiskit import Aer
from qiskit.quantum_info import state_fidelity, DensityMatrix, partial_trace

# QuantumCircuitの例
qc1 = qiskit.QuantumCircuit(5)
qc1.h(0)
qc1.cx(0, 1)

qc2 = qiskit.QuantumCircuit(7)
qc2.x(0)
qc2.h(1)

qc3 = qiskit.QuantumCircuit(8)
qc3.y(0)
qc3.y(1)

qc4 = qiskit.QuantumCircuit(2)
qc4.z(0)
qc4.h(1)

# 与えられたQuantumCircuitから最終状態ベクトルを取得する関数
def get_final_state(qc):
    backend = Aer.get_backend('statevector_simulator')
    job = qiskit.execute(qc, backend)
    return job.result().get_statevector(qc)

# 2番目のビットに対する部分フィデリティを計算する関数
def partial_fidelity(state1, state2, num_qubits1, num_qubits2):
    # 対象の量子ビット以外をトレースアウトする
    trace_out_qubits1 = list(set(range(num_qubits1)) - {1})
    trace_out_qubits2 = list(set(range(num_qubits2)) - {1})
    
    rho1 = partial_trace(DensityMatrix(state1), trace_out_qubits1)
    rho2 = partial_trace(DensityMatrix(state2), trace_out_qubits2)
    return state_fidelity(rho1, rho2)

# 部分フィデリティのテーブルを作成
circuits = [Nstep_before_AQCEL, Nstep_after_AQCEL, Optimized_before_AQCEL, Optimized_after_AQCEL]
names = ['NQPSB', 'NQPSA', 'OQPSB', 'OQPSA']
num_qubits_list = [5, 7, 8, 2]

states = [get_final_state(qc) for qc in circuits]

fidelities = [[partial_fidelity(states[i], states[j], num_qubits_list[i], num_qubits_list[j]) for j in range(4)] for i in range(4)]

# テーブルを表示
print("    ", end="")
for name in names:
    print(name, end="    ")
print()

for i in range(4):
    print(names[i], end=" ")
    for j in range(4):
        print("{:.4f}".format(fidelities[i][j]), end="  ")
    print()


NameError: name 'Nstep_before_AQCEL' is not defined